![](../img/330-banner.png)

# Tutorial 3

UBC 2024-25

## Outline

During this tutorial, we will focus on preprocessing - the necessary steps to perform to make the data meaningful for a learning algorithm.

All questions can be discussed with your classmates and the TAs - this is not a graded exercise!

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("../code/.")
from plotting_functions import *
from utils import *

pd.set_option("display.max_colwidth", 200)

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## `ColumnTransformer` on the California housing dataset 

In this notebook, you will practice features preprocessing using the [California housing dataset](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Let's start by loading the dataset (this is done for you):

In [2]:
housing_df = pd.read_csv("../data/housing.csv")
train_df, test_df = train_test_split(housing_df, test_size=0.1, random_state=123)

train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
6051,-117.75,34.04,22.0,2948.0,636.0,2600.0,602.0,3.1250,113600.0,INLAND
20113,-119.57,37.94,17.0,346.0,130.0,51.0,20.0,3.4861,137500.0,INLAND
14289,-117.13,32.74,46.0,3355.0,768.0,1457.0,708.0,2.6604,170100.0,NEAR OCEAN
13665,-117.31,34.02,18.0,1634.0,274.0,899.0,285.0,5.2139,129300.0,INLAND
14471,-117.23,32.88,18.0,5566.0,1465.0,6303.0,1458.0,1.8580,205000.0,NEAR OCEAN


Let's also add some new features that may help us with the prediction:

In [3]:
train_df = train_df.assign(
    rooms_per_household=train_df["total_rooms"] / train_df["households"]
)
test_df = test_df.assign(
    rooms_per_household=test_df["total_rooms"] / test_df["households"]
)

train_df = train_df.assign(
    bedrooms_per_household=train_df["total_bedrooms"] / train_df["households"]
)
test_df = test_df.assign(
    bedrooms_per_household=test_df["total_bedrooms"] / test_df["households"]
)

train_df = train_df.assign(
    population_per_household=train_df["population"] / train_df["households"]
)
test_df = test_df.assign(
    population_per_household=test_df["population"] / test_df["households"]
)

Finally, we are separating for you the target from the features:

In [4]:
# Let's keep both numeric and categorical columns in the data.
X_train = train_df.drop(columns=["median_house_value"])
y_train = train_df["median_house_value"]

X_test = test_df.drop(columns=["median_house_value"])
y_test = test_df["median_house_value"]

## Step 1

Your turn now! Start by importing ColumnTranformer and make_column_transformer

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_household,population_per_household
6051,-117.75,34.04,22.0,2948.0,636.0,2600.0,602.0,3.1250,INLAND,4.897010,1.056478,4.318937
20113,-119.57,37.94,17.0,346.0,130.0,51.0,20.0,3.4861,INLAND,17.300000,6.500000,2.550000
14289,-117.13,32.74,46.0,3355.0,768.0,1457.0,708.0,2.6604,NEAR OCEAN,4.738701,1.084746,2.057910
13665,-117.31,34.02,18.0,1634.0,274.0,899.0,285.0,5.2139,INLAND,5.733333,0.961404,3.154386
14471,-117.23,32.88,18.0,5566.0,1465.0,6303.0,1458.0,1.8580,NEAR OCEAN,3.817558,1.004801,4.323045
...,...,...,...,...,...,...,...,...,...,...,...,...
7763,-118.10,33.91,36.0,726.0,NaN,490.0,130.0,3.6389,<1H OCEAN,5.584615,NaN,3.769231
15377,-117.24,33.37,14.0,4687.0,793.0,2436.0,779.0,4.5391,<1H OCEAN,6.016688,1.017972,3.127086
17730,-121.76,37.33,5.0,4153.0,719.0,2435.0,697.0,5.6306,<1H OCEAN,5.958393,1.031564,3.493544
15725,-122.44,37.78,44.0,1545.0,334.0,561.0,326.0,3.8750,NEAR BAY,4.739264,1.024540,1.720859


## Step 2

Next, group features by type (numerical or categorical). You may also want to save the target separately. 

In [6]:
numeric_feats = ["longitude", "latitude", "housing_median_age", "median_income", "rooms_per_household", "bedrooms_per_household", "population_per_household"]
categorical_feats = ["ocean_proximity"]  # apply one-hot encoding
passthrough_feats = []  # do not apply any transformation
drop_feats = ["total_rooms", "total_bedrooms", "population", "households"]

## Step 3

Create a ColumnTransformer for your features. The transformer should include imputation and scaling for numeric features, and encoding for categorical features (which type of encoding?)

In [10]:
ct = make_column_transformer(    
    (
        make_pipeline(SimpleImputer(), StandardScaler()),
        numeric_feats,
    ),
    ("passthrough", passthrough_feats),
    (OneHotEncoder(), categorical_feats),
    ("drop", drop_feats),
)
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'median_income', 'rooms_per_household',
                                  'bedrooms_per_household',
                                  'population_per_household']),
                                ('passthrough', 'passthrough', []),
                                ('onehotencoder', OneHotEncoder(),
                                 ['ocean_proximity']),
                                ('drop', 'drop',
                                 ['total_rooms', 'total_bedrooms', 'population',
                                  'households'])])

## Step 4

Visualize the transformed training set as a dataframe

In [11]:
transformed = ct.fit_transform(X_train)
column_names = (
    numeric_feats
    + passthrough_feats    
    + ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
)
pd.DataFrame(transformed, columns=column_names)

,longitude,latitude,housing_median_age,median_income,rooms_per_household,bedrooms_per_household,population_per_household,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,0.908140,-0.743917,-0.526078,-0.389736,-0.210591,-8.481861e-02,0.126398,0.0,1.0,0.0,0.0,0.0
1,-0.002057,1.083123,-0.923283,-0.198924,4.726412,1.116619e+01,-0.050132,0.0,1.0,0.0,0.0,0.0
2,1.218207,-1.352930,1.380504,-0.635239,-0.273606,-2.639390e-02,-0.099240,0.0,0.0,0.0,0.0,1.0
3,1.128188,-0.753286,-0.843842,0.714077,0.122307,-2.813253e-01,0.010183,0.0,1.0,0.0,0.0,0.0
4,1.168196,-1.287344,-0.843842,-1.059242,-0.640266,-1.916284e-01,0.126808,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18571,0.733102,-0.804818,0.586095,-0.118182,0.063110,4.589354e-16,0.071541,1.0,0.0,0.0,0.0,0.0
18572,1.163195,-1.057793,-1.161606,0.357500,0.235096,-1.644065e-01,0.007458,1.0,0.0,0.0,0.0,0.0
18573,-1.097293,0.797355,-1.876574,0.934269,0.211892,-1.363136e-01,0.044029,1.0,0.0,0.0,0.0,0.0
18574,-1.437367,1.008167,1.221622,0.006578,-0.273382,-1.508311e-01,-0.132875,0.0,0.0,0.0,1.0,0.0


## Step 5

Finally, let's train a classifier (or even better, for practice, a baseline and another classifier): 
- create a pipeline with the preprocessor and a classifier of your choice.
- use the pipeline to perform cross-validation

In [13]:
pipe = make_pipeline(ct, SVC())
pipe.fit(X_train, y_train)
pipe.predict(X_test)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


array([500001., 137500., 500001., ..., 500001., 112500., 500001.])

## <font color='red'>Recap/comprehension questions</font>

- If we only plan to use a Decision Tree as classifier, do we still need to scale the numerical features?
- If the dataset included an ordinal feature "Neighbourhood desirability", with numerical labels 1 (poor), 2 (good) and 3 (excellent), would we need to apply an ordinal encoder to it?
- Why do we add the argument `drop="if_binary"` to `OneHotEncoder` when dealing with categorical features with only two possible values? What would be the disadvantages of not doing so?